In [2]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob

filename_queue=glob.glob('./rawdata/compdata[1-3].csv')


In [3]:
    #showing shape of the data
for file in filename_queue:
    df=pd.read_csv(file)
    print(file,df.shape)
#    for row in df:
#        print(row.shape())

./rawdata/compdata2.csv (21, 4503)
./rawdata/compdata1.csv (20, 4503)
